In [34]:
import requests
import shelve
import pandas as pd

ModuleNotFoundError: No module named 'requests'

In [22]:
params = {
    'url': 'http://localhost:3000/customer-information',
    'params': {'document': '09089356000118'}
}
resp = requests.get(**params).json()
resp

{'status_code': 200,
 'message': 'Success',
 'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
   'categoria_conta': 'grande',
   'classficacao_conta': 'E'},
  'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
   'quantidade_transacoes': 47,
   'saldo': 64061.48,
   'faturamento': 971786},
  'informacoes_bureau_credito': {'score_risco': 376,
   'pendencia_financeira': 0,
   'anotacoes_negativas': 0},
  'informacoes_receita_federal': {'cnpj': '09089356000118',
   'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
   'natureza_juridica': 'Sociedade Anônima Fechada',
   'qualificacao_responsavel': 'Diretor',
   'capital_social': 31301696.0,
   'porte': '',
   'ente_federativo_responsavel': None,
   'nome_fantasia': None,
   'situacao_cadastral': 'Ativa',
   'data_situacao_cadastral': '2007-09-27',
   'motivo_situacao_cadastral': 'SEM MOTIVO',
   'nome_da_cidade_no_exterior': None,
   'pais': None,
   'endereco': {'tipo_logradouro': 'AVENID

In [7]:
from tinydb import TinyDB, Query

# Abrir (ou criar) um banco de dados TinyDB
db = TinyDB('files/database/kvstore_tinydb.json')
KeyValue = Query()

In [44]:
data = {}
for key in db.get(KeyValue.analyses_id == 1).get('data').get('data').keys():
    data.update(db.get(KeyValue.analyses_id == 1).get('data').get('data').get(key))
data

{'abertura_conta': '2022-08-12',
 'categoria_conta': 'grande',
 'classficacao_conta': 'E',
 'media_transacoes_tres_meses': 2712.44,
 'quantidade_transacoes': 47,
 'saldo': 64061.48,
 'faturamento': 971786,
 'score_risco': 376,
 'pendencia_financeira': 0,
 'anotacoes_negativas': 0,
 'cnpj': '09089356000118',
 'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
 'natureza_juridica': 'Sociedade Anônima Fechada',
 'qualificacao_responsavel': 'Diretor',
 'capital_social': 31301696.0,
 'porte': '',
 'ente_federativo_responsavel': None,
 'nome_fantasia': None,
 'situacao_cadastral': 'Ativa',
 'data_situacao_cadastral': '2007-09-27',
 'motivo_situacao_cadastral': 'SEM MOTIVO',
 'nome_da_cidade_no_exterior': None,
 'pais': None,
 'endereco': {'tipo_logradouro': 'AVENIDA',
  'logradouro': 'PRESIDENTE JUSCELINO KUBITSCHEK',
  'numero': '31',
  'complemento': 'CONJ  A                   PAVMTO2 E 3',
  'bairro': 'BAUXITA',
  'cep': '35402179',
  'uf': 'MG',
  'municipio': 'OURO PRETO'},


In [39]:
def key_exists_any_level(data, key):
    if isinstance(data, list):
        return any(key_exists_any_level(item, key) for item in data)
    elif isinstance(data, dict):
        if key in data:
            return True
        return any(key_exists_any_level(value, key) for value in data.values())
    return False

# Função para verificar se a chave existe em qualquer nível do banco
def key_exists_in_db(key):
    return any(key_exists_any_level(entry, key) for entry in db.all())

exists = key_exists_in_db('classficacao_conta')
exists

True

In [47]:
rules = [
    {
        'rule': 'categoria_conta == "grande"',
        'code': 'REP001',
        'descricao': 'Categoria da conta Grande',
    },
    {
        'rule': 'classficacao_conta in ["E", "F"]',
        'code': 'REP001',
        'descricao': 'Categoria da conta Grande',
    }, 
    {
        'rule': 'bloqueio == True',
        'code': 'REP002',
        'descricao': 'Bloqueio na conta',
    }    
]

In [52]:
def decisao(rules, layer):
    decisao_rule = []
    errors = []
    for rule in rules:
        try:
            pd.eval(rule.get('rule'), local_dict=data)
            result = eval(rule.get('rule'), {}, data)
            if result:
                decisao_rule.append(rule)
        except Exception as err:
            decisao_rule_error = rule.copy()
            decisao_rule_error['error_detail'] = str(err)
            errors.append(decisao_rule_error)
    resultado_final = 'REPROVADO' if len(decisao_rule) > 0 else 'APROVADO'
    resultado_final = 'ERROR' if len(errors) > 0 else resultado_final
    decisao = {
        'processo': layer,
        'decisao': resultado_final,
        'codigos': decisao_rule,
    }
    if errors:
        decisao['errors'] = errors
    return decisao

In [53]:
decisao(rules, 'camada_1')

{'processo': 'camada_1',
 'decisao': 'ERROR',
 'codigos': [{'rule': 'categoria_conta == "grande"',
   'code': 'REP001',
   'descricao': 'Categoria da conta Grande'},
  {'rule': 'classficacao_conta in ["E", "F"]',
   'code': 'REP001',
   'descricao': 'Categoria da conta Grande'}],
 'errors': [{'rule': 'bloqueio == True',
   'code': 'REP002',
   'descricao': 'Bloqueio na conta',
   'error_detail': "name 'bloqueio' is not defined"}]}

In [32]:
db.insert({'item': 'request_data', 'analyses_id': 2, 'data': resp})

2

In [35]:
db.search(KeyValue.analyses_id == 2)

[{'item': 'request_data',
  'analyses_id': 2,
  'data': {'status_code': 200,
   'message': 'Success',
   'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
     'categoria_conta': 'grande',
     'classficacao_conta': 'E'},
    'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
     'quantidade_transacoes': 47,
     'saldo': 64061.48,
     'faturamento': 971786},
    'informacoes_bureau_credito': {'score_risco': 376,
     'pendencia_financeira': 0,
     'anotacoes_negativas': 0},
    'informacoes_receita_federal': {'cnpj': '09089356000118',
     'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
     'natureza_juridica': 'Sociedade Anônima Fechada',
     'qualificacao_responsavel': 'Diretor',
     'capital_social': 31301696.0,
     'porte': '',
     'ente_federativo_responsavel': None,
     'nome_fantasia': None,
     'situacao_cadastral': 'Ativa',
     'data_situacao_cadastral': '2007-09-27',
     'motivo_situacao_cadastral': 'SEM MOTIVO'

In [30]:
db.all()

[{'item': 'request_data',
  'analyses_id': 1,
  'data': {'status_code': 200,
   'message': 'Success',
   'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
     'categoria_conta': 'grande',
     'classficacao_conta': 'E'},
    'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
     'quantidade_transacoes': 47,
     'saldo': 64061.48,
     'faturamento': 971786},
    'informacoes_bureau_credito': {'score_risco': 376,
     'pendencia_financeira': 0,
     'anotacoes_negativas': 0},
    'informacoes_receita_federal': {'cnpj': '09089356000118',
     'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
     'natureza_juridica': 'Sociedade Anônima Fechada',
     'qualificacao_responsavel': 'Diretor',
     'capital_social': 31301696.0,
     'porte': '',
     'ente_federativo_responsavel': None,
     'nome_fantasia': None,
     'situacao_cadastral': 'Ativa',
     'data_situacao_cadastral': '2007-09-27',
     'motivo_situacao_cadastral': 'SEM MOTIVO'

In [26]:


# Abrir (ou criar) um banco de dados shelve
with shelve.open('files/database/responses.db') as db:
    # db['item'] = 'request_data'
    # db['analyses_id'] = 2
    # db['data'] = resp
    dicionario = {key: value for key, value in db.items()}
dicionario

{'item': 'request_data',
 'analyses_id': 2,
 'data': {'status_code': 200,
  'message': 'Success',
  'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
    'categoria_conta': 'grande',
    'classficacao_conta': 'E'},
   'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
    'quantidade_transacoes': 47,
    'saldo': 64061.48,
    'faturamento': 971786},
   'informacoes_bureau_credito': {'score_risco': 376,
    'pendencia_financeira': 0,
    'anotacoes_negativas': 0},
   'informacoes_receita_federal': {'cnpj': '09089356000118',
    'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
    'natureza_juridica': 'Sociedade Anônima Fechada',
    'qualificacao_responsavel': 'Diretor',
    'capital_social': 31301696.0,
    'porte': '',
    'ente_federativo_responsavel': None,
    'nome_fantasia': None,
    'situacao_cadastral': 'Ativa',
    'data_situacao_cadastral': '2007-09-27',
    'motivo_situacao_cadastral': 'SEM MOTIVO',
    'nome_da_cidade_no_

In [18]:
db.keys()

KeysView(<shelve.DbfilenameShelf object at 0x000001B97D407B30>)

In [19]:
{key: value for key, value in db.items()}

ValueError: invalid operation on closed shelf